In [1]:
import pandas as pd

# Đọc dữ liệu
faculty_info = pd.read_csv('faculty_info.csv')
course_survey = pd.read_csv('course_survey.csv')
student_comments = pd.read_csv('student_comments.csv')

# === SỬA LỖI CỘT rating_score ===
# Loại bỏ khoảng trắng, thay dấu phẩy bằng dấu chấm, chuyển sang float
course_survey['rating_score'] = (
    course_survey['rating_score']
    .astype(str)                  # Đảm bảo là string để xử lý
    .str.strip()                  # Xóa khoảng trắng đầu/cuối
    .str.replace(',', '.')        # Thay 8,5 → 8.5
    .replace('', None)            # Nếu có ô trống → None
    .astype(float)                # Chuyển sang float
)

# Kiểm tra nhanh
print("rating_score sau khi sửa:")
print(course_survey['rating_score'].head(10))
print("Kiểu dữ liệu:", course_survey['rating_score'].dtype)  # Phải là float64

# Nếu có giá trị NaN (do dữ liệu lỗi), có thể fill hoặc drop tùy yêu cầu
# course_survey['rating_score'].fillna(0, inplace=True)  # hoặc dropna()

# Tiếp tục merge như cũ
df_survey = course_survey.merge(faculty_info, on='faculty_id', how='left')
df = df_survey.merge(student_comments, on='survey_id', how='left')

# Xử lý sentiment nếu cần
df['sentiment_label'].fillna('Trung lập', inplace=True)

print("\nDữ liệu sau merge:")
print(df.head())

rating_score sau khi sửa:
0    3.5
1    7.0
2    3.5
3    7.0
4    3.5
5    8.0
6    3.5
7    4.0
8    4.0
9    8.5
Name: rating_score, dtype: float64
Kiểu dữ liệu: float64

Dữ liệu sau merge:
  survey_id faculty_id course_code student_level  rating_score survey_term  \
0   SRV1000      GV106       DA200        nam  2           3.5     2024-2    
1   SRV1000      GV106       DA200        nam  2           3.5     2024-2    
2   SRV1001      GV120      CT 102    Liên thông           7.0   HK1- 2024   
3   SRV1002      GV111       CT101    Liên thông           3.5   HK1- 2024   
4   SRV1003      gv123       CT101           K18           7.0     2024-2    

        full_name                department            comment_text  \
0    Nguyen van A               bo mon toan  Giang vien day de hieu   
1    Nguyen van A               bo mon toan     Bài giảng hơi nhanh   
2    Tran  Thi  B  KHOA CONG NGHE THONG TIN                     NaN   
3     LE   VAN  C                 khoa cntt           

C:\Users\QUYEN\AppData\Local\Temp\ipykernel_9820\1500212003.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['sentiment_label'].fillna('Trung lập', inplace=True)


In [2]:
### Nhiệm vụ 3: GroupBy & Tổng hợp

# 1. Rating trung bình theo giảng viên
avg_by_faculty = df.groupby(['faculty_id', 'full_name', 'department'], as_index=False).agg(
    avg_rating=('rating_score', 'mean'),
    num_surveys=('rating_score', 'count')
).round({'avg_rating': 2}).sort_values('avg_rating', ascending=False)

print("\n1. Rating trung bình theo giảng viên (sắp xếp giảm dần):")
print(avg_by_faculty)

# Các phần còn lại (theo department, pivot, sentiment...) sẽ chạy bình thường


1. Rating trung bình theo giảng viên (sắp xếp giảm dần):
   faculty_id       full_name                department  avg_rating  \
5       GV115    Nguyen van A                 khoa cntt        7.75   
6       GV116     LE   VAN  C  KHOA CONG NGHE THONG TIN        7.00   
17      GV118     pham  thi d                 khoa cntt        6.80   
14     GV107      LE   VAN  C              Khoa Kinh tế        6.50   
20      gv101   Hoang   van e              Khoa Kinh tế        6.50   
11     GV102      LE   VAN  C               Bộ môn Toán        6.00   
3       GV111     LE   VAN  C                 khoa cntt        5.95   
15     GV112    Hoang   van e               bo mon toan        5.71   
4       GV113    Nguyen van A               Bo mon Toan        5.67   
8       GV120    Tran  Thi  B  KHOA CONG NGHE THONG TIN        5.50   
2       GV110    Nguyen van A               Bo mon Toan        5.07   
12      GV103   Hoang   van e             khoa kinh  te        5.00   
22      gv119     L

In [4]:
# 2. Rating trung bình theo department
avg_by_dept = (
    df.groupby('department')['rating_score']
    .mean()
    .round(2)
    .sort_values(ascending=False)
    .reset_index(name='avg_rating')
)

print("\n2. Rating trung bình theo khoa/bộ môn:")
print(avg_by_dept)


2. Rating trung bình theo khoa/bộ môn:
                 department  avg_rating
0                 khoa cntt        6.18
1               Bộ môn Toán        6.00
2              Khoa Kinh tế        5.95
3  KHOA CONG NGHE THONG TIN        5.19
4               bo mon toan        4.97
5             khoa kinh  te        4.85
6              Khoa Kinh te        4.82
7               Bo mon Toan        4.67
8                 Khoa CNTT        4.27


In [5]:
# 3. Rating trung bình theo department × student_level (pivot)
avg_by_dept_level = (
    df.groupby(['department', 'student_level'])['rating_score']
    .mean()
    .round(2)
    .unstack(fill_value=0)
)

print("\n3. Rating trung bình theo department x student_level:")
print(avg_by_dept_level)



3. Rating trung bình theo department x student_level:
student_level              K17   K18  Lien thong  Liên thông  Nam 1  Năm 1  \
department                                                                   
Bo mon Toan               8.50  6.83        4.38        3.50   0.00   4.60   
Bộ môn Toán               4.50  3.50        8.00        0.00   0.00   0.00   
KHOA CONG NGHE THONG TIN  0.00  5.50        0.00        7.50   3.50   4.00   
Khoa CNTT                 4.50  4.50        0.00        4.70   3.67   0.00   
Khoa Kinh te              4.50  5.75        0.00        4.50   4.25   7.00   
Khoa Kinh tế              7.50  4.00        5.88        3.50   5.57   8.00   
bo mon toan               4.75  3.50        0.00        4.50   8.00   4.62   
khoa cntt                 6.50  6.00        6.62        4.25   8.50   5.75   
khoa kinh  te             4.00  0.00        0.00        4.50   5.25   4.50   

student_level             Năm 2  k17   nam  2  
department                            

In [6]:
# 4. Tỷ lệ bình luận tích cực/tiêu cực/trung lập theo department
sentiment_counts = df.groupby(['department', 'sentiment_label']).size().unstack(fill_value=0)

# Đảm bảo có đủ 3 cột
for label in ['Tích cực', 'Tiêu cực', 'Trung lập']:
    if label not in sentiment_counts.columns:
        sentiment_counts[label] = 0

sentiment_counts['Total'] = sentiment_counts.sum(axis=1)
sentiment_counts['Tích cực (%)'] = (sentiment_counts['Tích cực'] / sentiment_counts['Total'] * 100).round(1)
sentiment_counts['Tiêu cực (%)'] = (sentiment_counts['Tiêu cực'] / sentiment_counts['Total'] * 100).round(1)
sentiment_counts['Trung lập (%)'] = (sentiment_counts['Trung lập'] / sentiment_counts['Total'] * 100).round(1)

sentiment_ratio = sentiment_counts[['Tích cực (%)', 'Tiêu cực (%)', 'Trung lập (%)']].sort_values('Tích cực (%)', ascending=False)

print("\n4. Tỷ lệ sentiment theo department:")
print(sentiment_ratio)


4. Tỷ lệ sentiment theo department:
sentiment_label           Tích cực (%)  Tiêu cực (%)  Trung lập (%)
department                                                         
khoa kinh  te                     17.6           5.9           35.3
bo mon toan                       11.8          17.6           41.2
Khoa Kinh te                       9.1           0.0           63.6
Khoa CNTT                          9.1           0.0           72.7
KHOA CONG NGHE THONG TIN           7.7           7.7           69.2
khoa cntt                          7.1           7.1           50.0
Khoa Kinh tế                       5.3          10.5           52.6
Bo mon Toan                        4.2           0.0           75.0
Bộ môn Toán                        0.0          25.0           25.0


In [7]:
# 5. Giảng viên rating cao nhất và thấp nhất (có ít nhất 5 khảo sát để đáng tin cậy)
min_surveys = 5

top_faculty = avg_by_faculty[avg_by_faculty['num_surveys'] >= min_surveys].head(10)
low_faculty = avg_by_faculty[avg_by_faculty['num_surveys'] >= min_surveys].sort_values('avg_rating').head(10)

print(f"\n5. Top 10 giảng viên rating cao nhất (≥ {min_surveys} khảo sát):")
print(top_faculty[['full_name', 'department', 'avg_rating', 'num_surveys']])

print(f"\nTop 10 giảng viên rating thấp nhất (≥ {min_surveys} khảo sát):")
print(low_faculty[['full_name', 'department', 'avg_rating', 'num_surveys']])

# Lưu kết quả ra file nếu cần
avg_by_faculty.to_csv('avg_rating_by_faculty.csv', index=False)
avg_by_dept.to_csv('avg_rating_by_department.csv', index=False)
avg_by_dept_level.to_csv('avg_rating_dept_level_pivot.csv')
sentiment_ratio.to_csv('sentiment_ratio_by_dept.csv')


5. Top 10 giảng viên rating cao nhất (≥ 5 khảo sát):
         full_name                department  avg_rating  num_surveys
17     pham  thi d                 khoa cntt        6.80           10
14     LE   VAN  C              Khoa Kinh tế        6.50           10
20   Hoang   van e              Khoa Kinh tế        6.50            5
3      LE   VAN  C                 khoa cntt        5.95           11
15   Hoang   van e               bo mon toan        5.71            7
8     Tran  Thi  B  KHOA CONG NGHE THONG TIN        5.50            5
2     Nguyen van A               Bo mon Toan        5.07            7
22     LE   VAN  C              Khoa Kinh te        5.00            5
7     Tran  Thi  B             khoa kinh  te        4.95           11
1    Hoang   van e                 khoa cntt        4.80            5

Top 10 giảng viên rating thấp nhất (≥ 5 khảo sát):
         full_name                department  avg_rating  num_surveys
23     LE   VAN  C               Bo mon Toan        4.